In [ ]:
# !pip install plotly matplotlib pandas requests
# !git clone https://github.com/HuobiRDCenter/huobi_Python
# !cd huobi_Python && python setup.py install

In [ ]:
import datetime
import requests
import pandas as pd
import random
from utils import *
from strategies import *
from matplotlib import pyplot as plt

In [ ]:
symbol_list = get_symbol_list()
sb = symbol_list[random.randint(0, len(symbol_list))]
for sb in symbol_list:
    a=get_price(sb, "30min", 2000)
    a=get_rate(a, "close")
    a["cr+1"] = (a["close"].shift(-1) - a["close"]) / a["close"]
    a["cr+2"] = (a["close"].shift(-2) - a["close"]) / a["close"]
    a=get_rate(a, "volume")
    a=custom_data_analyze_func(a, rolling_days=3)
    standard = a.describe()["vol_mean3_cr"]["75%"]
    c=a.loc[(a["vol_mean3_cr"]>standard) & (a["close_rate"]>0)]
    try:
        if datetime.datetime.strptime("2021-04-22 17:00:00", "%Y-%m-%d %H:%M:%S") == c.index[-1]:
            print(sb)
    except:
        continue
print("done")

In [ ]:
# symbol_list = get_symbol_list()
# sb = symbol_list[random.randint(0, len(symbol_list))]
sb = "mtausdt"
a=get_price(sb, "30min", 2000)
print(sb)
a=get_rate(a, "close")
a["cr+1"] = (a["close"].shift(-1) - a["close"]) / a["close"]
a["cr+2"] = (a["close"].shift(-2) - a["close"]) / a["close"]
a=get_rate(a, "volume")
a=custom_data_analyze_func(a, rolling_days=3)
standard = a.describe()["vol_mean3_cr"]["75%"]
c=a.loc[(a["vol_mean3_cr"]>standard) & (a["close_rate"]>0)]
print(c.index[-1], c.iloc[-1]["close"])
display(c[["cr+1","cr+2","close_rate"]].describe())
# c[["cr+1"]].plot(kind="hist")
aa=get_price(sb, "1min", 2000)
aa=get_mean_line(aa)
bb = aa[c.index[-1]-datetime.timedelta(hours=0.5) : c.index[-1]]
draw_candle(bb)
cc = aa[c.index[-1]:c.index[-1]+datetime.timedelta(hours=0.5)]
draw_candle(cc)

In [ ]:
wallet = 50 # USDT

In [ ]:
import datetime
from utils import *
from plotly import graph_objects as go
from IPython.display import clear_output

pr = 0
max_rate = 0
stime = datetime.datetime.now()

plot_list = []

#TODO 从服务器获取当前持仓信息
symbol = "dogeusdt"
open_point=get_price(symbol, "1min", 1).iloc[-1]["close"]
leverage = 30
direction = "b"
init_win_rates = 20
force_close = -20
change_rate = 10

while True:
    try:
        current_point = get_price(symbol, "1min", 1).iloc[-1]["close"]
    except Exception:
        print("!!!!!!")
        continue
    if direction in "buy":
        rates = (current_point - open_point) / open_point * 100 * leverage
    elif direction in "sell":
        rates = (open_point - current_point) / open_point * 100 * leverage
    else:
        raise Exception("wha???????")
    if pr != rates:
        clear_output(wait=True)
        if max_rate < rates:
            max_rate = rates
        duration = datetime.datetime.now()-stime
        print(f"It is {duration} later")
        print("current rate = {:.4f}%".format(rates))
        print("================================")
        print("The max rate: {:.4f}%".format(max_rate))
        print(f"U {direction} {symbol} with {open_point}\nThe force close line: {force_close}\nThe win rates: {init_win_rates}")
        plot_list.append(rates)
    pr = rates
    if rates < force_close:
        if rates < 0:
            print("I quite QAQ")
            wallet = wallet * (100+rates)/100
        else:
            print(f"I quite XD, with {rates}%")
            wallet = wallet * (100+rates)/100
        break
    if rates > 2* init_win_rates:
        print(f"I quite XD, with {rates}%")
        wallet = wallet * (100+rates)/100
    elif rates > init_win_rates*0.9:
        force_close = init_win_rates - change_rate
        init_win_rates += change_rate

print(f"{wallet} usdt left")
        
fig = go.Figure(
    go.Scatter(y=plot_list)
)
fig.show()

In [ ]:
from huobi.client.account import AccountClient
from huobi.client.trade import TradeClient
from huobi.constant import OrderType
print(OrderType.BUY_MARKET)


ids =[account.id for account in ac.get_accounts()]

blc = ac.get_balance(ids[0])
[b.balance for b in ac.get_balance(ids[1])]